In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql

# Connect to the SQLite database
conn = sql.connect('/kaggle/input/movie-data-analytics-dataset/movie.sqlite')

# Create a cursor
cursor = conn.cursor()

# Execute a SQL query to retrieve table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all table names
table_names = cursor.fetchall()

# Print the table names
print("Tables in the database:")
for table in table_names:
    print(table[0])

### Lets count how many rows are there for each table:

In [ ]:
for table in table_names:
    table_name = table[0]
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    row_count = cursor.fetchone()[0]
    print(f"Table: {table_name}, Row Count: {row_count}")

### Now let's see what are the columns for each table:

In [ ]:
table_names = ['IMDB', 'earning', 'genre']

for name in table_names:
    columns = []
    
    # Execute a PRAGMA statement to retrieve column names for a table
    cursor.execute(f"PRAGMA table_info({name})")

    # Fetch all rows, each containing information about a column
    column_info = cursor.fetchall()

    # Extract column names from the result
    column_names = [info[1] for info in column_info]

    # Print the column names
    for col in column_names:
        columns.append(col)
    print(f"Column names for {name}:\n {columns}\n")

### The common column for each table is "Movie_id", which we will use to join the tables and generate a pandas DataFrame

In [ ]:
query = 'SELECT * from IMDB'

imdb = pd.read_sql_query(query, conn)

In [ ]:
query = '''
SELECT *
FROM IMDB
JOIN earning ON IMDB.Movie_id = earning.Movie_id
JOIN genre ON IMDB.Movie_id = genre.Movie_id
'''

df = pd.read_sql_query(query, conn)

df.head(1)

In [ ]:
print(df.columns, df.shape)

In [ ]:
df.duplicated('Title').sum()

In [ ]:
imdb.duplicated('Title').sum()

#### There are 234 rows duplicated in the DataFrame. 
#### That is because the 'genre' table has many genres for a unique Movie_id (a many to one type of relationship)
#### 
#### To correct this, let's make a new query concatenating the genres of the 'genre' table in a single column for the new DataFrame:

In [ ]:
query = '''
SELECT IMDB.*, earning.*, GROUP_CONCAT(genre.Genre, ', ') AS Genres
FROM IMDB
JOIN earning ON IMDB.Movie_id = earning.Movie_id
JOIN genre ON IMDB.Movie_id = genre.Movie_id
GROUP BY IMDB.Movie_id;
'''

df = pd.read_sql_query(query, conn)

df.head(2)

## Ok, now we can begin our Exploratory Data Analysis (EDA)

In [ ]:
df.info()